# Openai Function Call

In [11]:
import os
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt

GPT_MODEL = "gpt-3.5-turbo-0613"

In [12]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [13]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [14]:
messages = []
messages.append({"role": "system", "content": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': 'Sure, can you please provide me the name of the city and state where you want to know the weather?'}

In [15]:
messages.append({"role": "user", "content": "I'm in Ahmedabad, Gujarat."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_RiccIR47Miom30JPyOvxNx5Z',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Ahmedabad, Gujarat",\n  "format": "celsius"\n}'}}]}

In [16]:
messages = []
messages.append({"role": "system", "content": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Ahemdabad, Gujarat over the next 3 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_n3WfKutPYuBP1c75VFBa5UfM',
   'type': 'function',
   'function': {'name': 'get_n_day_weather_forecast',
    'arguments': '{\n  "location": "Ahmedabad, Gujarat",\n  "format": "celsius",\n  "num_days": 3\n}'}}]}

In [17]:
messages = []
messages.append({"role": "system", "content": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_0DX58RWYKgQPHPJ5dfrPvH5O',
   'type': 'function',
   'function': {'name': 'get_n_day_weather_forecast',
    'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}'}}]}

In [18]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_sYHqzCCPsT9pk8SoHB35ho2m',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}}]}

In [19]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}

In [20]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model='gpt-3.5-turbo-1106'
)

chat_response.json()
assistant_message = chat_response.json()["choices"][0]["message"]['tool_calls']
assistant_message


[{'id': 'call_Mgo4dD5e457dQ1zm85cdOkG8',
  'type': 'function',
  'function': {'name': 'get_n_day_weather_forecast',
   'arguments': '{"location": "San Francisco, CA", "format": "celsius", "num_days": 4}'}},
 {'id': 'call_u3jGrJzybPoO0FujNNnTkr2P',
  'type': 'function',
  'function': {'name': 'get_n_day_weather_forecast',
   'arguments': '{"location": "Glasgow", "format": "celsius", "num_days": 4}'}}]